<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/trying_french_monolingual_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
pip install transformers

     |████████████████████████████████| 1.8MB 5.1MB/s 
     |████████████████████████████████| 2.9MB 17.4MB/s 
     |████████████████████████████████| 890kB 20.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8891652081e031c023f8cacfd398237c2bcdba635c1c517ba032dc5bdb6b2110
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.8MB/s 


In [19]:
pip install wandb

     |████████████████████████████████| 1.9MB 4.3MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 133kB 35.3MB/s 
     |████████████████████████████████| 163kB 31.2MB/s 
     |████████████████████████████████| 102kB 8.6MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6490 sha256=ec0ddb1b5b461583e87a20f77b777eb12e0bc8948115502b194845ecc518bb11
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74842 sha256=876f91c524185005ac82922599def346b2067c87a12a7e8dff52be3e93713011
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=6919446222e38a2e735507576e7204f9f54fd5e0545468

In [41]:
# %%writefile code_counts.py

import logging
import os
import random
import time
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

import numpy as np
import torch
from torch.utils.data.dataset import Dataset, IterableDataset
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoTokenizer, EvalPrediction, Trainer, HfArgumentParser, TrainingArguments, \
    AutoModelForSequenceClassification, set_seed, AutoConfig
from transformers import PreTrainedTokenizer, DataCollator, PreTrainedModel
import wandb

import pandas as pd
def load_data():
    trn=pd.read_csv('/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train.csv.zip',usecols=['toxic','comment_text'])
    tst=pd.read_csv('/content/gdrive/My Drive/multilingual/test.csv.zip',usecols=['lang','content'])  
    sub=pd.read_csv('/content/gdrive/My Drive/multilingual/submission.csv') 
    val=pd.read_csv( '/content/gdrive/My Drive/multilingual/validation.csv.zip') 
    return trn,tst,sub,val

def get_lang(val,sub,test,lang):
  output=pd.concat([val.loc[val['lang']==lang],test.loc[test['lang']==lang]]).reset_index(drop=True)
  output=output.drop(['id','lang'],1)
  return output

def build_batches(sentences, batch_size):
    batch_ordered_sentences = list()
    while len(sentences) > 0:
        to_take = min(batch_size, len(sentences))
        select = random.randint(0, len(sentences) - to_take)
        batch_ordered_sentences += sentences[select:select + to_take]
        del sentences[select:select + to_take]
    return batch_ordered_sentences
class TextDataset(Dataset):
    def __init__(self, tokenizer, pad_to_max_length, max_len,
                 examples) :
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.examples: List[Example] = examples
        self.current = 0
        self.pad_to_max_length = pad_to_max_length

    def encode(self, ex) :
        encode_dict = self.tokenizer.encode_plus(text=ex.text_a,
                                                 text_pair=ex.text_b,
                                                 add_special_tokens=True,
                                                 max_length=self.max_len,
                                                 pad_to_max_length=self.pad_to_max_length,
                                                 return_token_type_ids=False,
                                                 return_attention_mask=True,
                                                 return_overflowing_tokens=False,
                                                 return_special_tokens_mask=False,
                                                 )
        return Features(input_ids=encode_dict["input_ids"],
                        attention_mask=encode_dict["attention_mask"],
                        label=ex.label)

    def __getitem__(self, idx) :  # Trainer doesn't support IterableDataset (define sampler)
        if self.current == len(self.examples):
            self.current = 0
        ex = self.examples[self.current]
        self.current += 1
        return self.encode(ex=ex)

    def __len__(self):
        return len(self.examples)
def load_transformers_model(pretrained_model_name_or_path,
                            use_cuda,
                            mixed_precision):
    config = AutoConfig.from_pretrained(pretrained_model_name_or_path=pretrained_model_name_or_path,
                                        num_labels=3)
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        config=config)
    if use_cuda and torch.cuda.is_available():
        device = torch.device('cuda')
        model.to(device)

    if mixed_precision:
        try:
            from apex import amp
            model = amp.initialize(model, opt_level='O1')
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    return model
class MyTrainer(Trainer):
    def _setup_wandb(self):
        wandb.init(project="speed_training",
                   config=vars(self.args),
                   name=self.args.output_dir)
        wandb.watch(self.model, log="gradients", log_freq=self.args.logging_steps)


def compute_metrics(p: EvalPrediction)  :
        preds = np.argmax(p.predictions, axis=1)
        return {"acc": (preds == p.label_ids).mean()}


@dataclass
class ModelParameters:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )
    dynamic_padding: bool = field(
        default=False,
        metadata={"help": "limit pad size at batch level"},
    )
    smart_batching: bool = field(
        default=False,
        metadata={"help": "build batch of similar sizes"},
    )
    dynamic_batch_size: bool = field(
        default=False,
        metadata={"help": "build batch of similar sizes"},
    )
class SmartCollator(DataCollator):
    pad_token_id: int

    def collate_batch(self, batch: List[Features]) -> Dict[str, torch.Tensor]:
        batch_inputs = list()
        batch_attention_masks = list()
        labels = list()
        max_size = max([len(ex.input_ids) for ex in batch])
        for item in batch:
            batch_inputs += [pad_seq(item.input_ids, max_size, self.pad_token_id)]
            batch_attention_masks += [pad_seq(item.attention_mask, max_size, 0)]
            labels.append(item.label)

        return {"input_ids": torch.tensor(batch_inputs, dtype=torch.long),
                "attention_mask": torch.tensor(batch_attention_masks, dtype=torch.long),
                "labels": torch.tensor(labels, dtype=torch.long)
                }

if __name__ == "__main__":
    # parser = HfArgumentParser((TrainingArguments, ModelParameters))
    # training_args, model_args = parser.parse_args_into_dataclasses()  # type: (TrainingArguments, ModelParameters)

 
    train,test,sub,val=load_data()
    test.columns=['comment_text','lang']
    test['toxic']=sub['toxic']

    french=get_lang(val,sub,test,'fr')
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="camembert-base")
    max_sequence_len=192

    train_batches = build_batches(sentences=list(french.comment_text.values), batch_size=16*8)










    train_set = TextDataset(tokenizer=tokenizer,
                            max_len=max_sequence_len,
                            examples=train_batches,
                            pad_to_max_length=True)





    model = load_transformers_model(pretrained_model_name_or_path="camembert-base",
                                    use_cuda=True,
                                    mixed_precision=False)








    trainer = MyTrainer(
        model=model,
        # args=training_args,
        train_dataset=train_set,
        # data_collator=IdentityCollator(pad_token_id=tokenizer.pad_token_id),
        data_collator=SmartCollator(pad_token_id=tokenizer.pad_token_id),
        tb_writer=SummaryWriter(log_dir='logs', flush_secs=10),
        eval_dataset=valid_set,
        compute_metrics=compute_metrics,
      )
    

    start_time = time.time()
    trainer.train()
    wandb.config.update(model_args)
    wandb.config.update(training_args)
    wandb.log({"training time": int((time.time() - start_time) / 60)})
    trainer.save_model()
    trainer.evaluate()
    logging.info("*** Evaluate ***")
    result = trainer.evaluate()
    wandb.log(result)

    output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logging.info("***** Eval results *****")
        for key, value in result.items():
            logging.info("  %s = %s", key, value)
            writer.write("%s = %s\n" % (key, value))

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

NameError: ignored

In [40]:
!python code_counts.py \
 			--output_dir ./models/speed_camembert_max_len_128_fp16_dynamic_padding_smart_batching_batch_64_seed_321 \
 			--overwrite_output_dir \
 			--save_steps 0 \
 			--seed 321 \
 			--num_train_epochs 1 \
 			--learning_rate 5e-5 \
 			--per_gpu_train_batch_size 64 \
 			--gradient_accumulation_steps 1 \
 			--per_gpu_eval_batch_size 64 \
 			--max_seq_len 128 \
 			--dynamic_padding \
 			--smart_batching \
 			--fp16 \
 			--evaluate_during_training ; \
	)

/bin/bash: -c: line 0: syntax error near unexpected token `)'
/bin/bash: -c: line 0: `python code_counts.py  			--output_dir ./models/speed_camembert_max_len_128_fp16_dynamic_padding_smart_batching_batch_64_seed_321  			--overwrite_output_dir  			--save_steps 0  			--seed 321  			--num_train_epochs 1  			--learning_rate 5e-5  			--per_gpu_train_batch_size 64  			--gradient_accumulation_steps 1  			--per_gpu_eval_batch_size 64  			--max_seq_len 128  			--dynamic_padding  			--smart_batching  			--fp16  			--evaluate_during_training ; 	)'


In [12]:
val.head()

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0


In [11]:
sub.head()

,toxic,id
0,0.000081,0
1,0.003086,1
2,0.308321,2
3,0.000055,3
4,0.000143,4


In [14]:
test.head()

,comment_text,lang,toxic
0,Doctor Who adlı viki başlığına 12. doctor olar...,tr,0.000081
1,"Вполне возможно, но я пока не вижу необходимо...",ru,0.003086
2,"Quindi tu sei uno di quelli conservativi , ...",it,0.308321
3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr,0.000055
4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr,0.000143


In [8]:
train.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.layers import *
from fastai.text import *
import multifit

In [ ]:
es=test.loc[test['lang']=='es'].reset_index(drop=True)
es.head()

,content,lang,toxic
0,el skate es unos de los deportes favoritos de ...,es,0.0
1,Me doy la bienvenida. A este usuari le gusta c...,es,0.0
2,"ES NOTABLEMENTE TENDENCIOSO, NO SE HABLA DE CU...",es,0.0
3,"El Jardín de infantes Nº938, fundado en 1989, ...",es,0.0
4,Daré explicaciones y/o aclaraciones a cualquie...,es,0.0


In [ ]:
def set_seed(seed):
    if seed is not None:
        torch.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)

In [ ]:
exp = multifit.from_pretrained("es_multifit_paper_version")
tokenizer = exp.pretrain_lm.tokenizer
fa_config =  tokenizer.get_fastai_config(add_open_file_processor=True)
data_lm = (TextList.from_df(es, **fa_config)
            .split_by_rand_pct(0.1)
            .label_for_lm()           
            .databunch(bs=32))
exp.finetune_lm.arch.lang = 'es'
set_seed(42)
learn = exp.finetune_lm.get_learner(data_lm)  
    # learn is a preconfigured fastai learner with a pretrained model loaded
data_lm.show_batch()

Training args:  {'drop_mult': 0.3, 'true_wd': False, 'wd': 1e-07, 'pretrained': False, 'clip': 0.12} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15, 'tie_weights': True, 'out_bias': True}
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Setting LM training seed seed to 0
Loading pretrained weights:  [PosixPath('/root/.fastai/models/es_multifit_paper_version/lm_best'), PosixPath('/root/.fastai/models/es_multifit_paper_version/itos')]


/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx_min = (t != self.pad_idx).nonzero().min()


idx,text
0,"▁, ▁ xxup ▁no ▁ xxup ▁se ▁ xxup ▁habla ▁ xxup ▁de ▁ xxup ▁cuando ▁ xxup ▁la ▁ xxup ▁tu pac ▁ xxup ▁amenaza ba ▁y ▁ xxup ▁fue ▁ xxup ▁denuncia do ▁ xxup ▁por ▁3 ▁ xxup ▁fiscal es ▁ xxup ▁en ▁ xxup ▁febrero ▁, ▁ xxup ▁ni ▁ xxup ▁de ▁ xxup ▁cama ras ▁ xxup ▁oculta s ▁, ▁ xxup ▁ni ▁ xxup"
1,▁para ▁que ▁lo ▁cu bri eran ▁en ▁sus ▁fe cho ría s ▁y ▁ahora ▁dice ▁su ▁escrito ▁que ▁es ▁un ▁héroe ▁nacional ▁. ▁ xxmaj ▁band ido ▁de ▁7 ▁su ela s ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁ cre o ▁que ▁después ▁de ▁mi ▁posterior ▁ edi t ▁todo ▁quedó ▁bien ▁. ▁ xxmaj ▁gracias ▁y ▁salud os ▁. ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁se ▁ha
2,"os ▁. ▁ xxmaj ▁pues ▁bueno ▁, ▁inicia ndo ▁con ▁el ▁tema ▁, ▁muy ▁di fici l mente ▁ cre o ▁que ▁no ▁se ▁haya ▁dado ▁cuenta ▁del ▁tiempo ▁de ▁su ▁bloqueo ▁puesto ▁que ▁siempre ▁, ▁cuando ▁inicia s ▁sesión ▁e ▁intenta s ▁ edi tar ▁, ▁el ▁sistema ▁ te ▁ avi sa ▁de ▁cu án to ▁tiempo ▁es ▁el ▁bloqueo ▁y ▁por ▁qué ▁razones ▁. ▁ xxmaj ▁en"
3,xxmaj ▁es ▁una ▁traducción ▁de ▁http ▁: ▁/ ▁/ ▁www . ri bb on s o ft . com ▁/ ▁ q ca d . html . ▁ xxmaj ▁no ▁esto y ▁seguro ▁que ▁está ▁bajo ▁ xxup ▁ g f d l ▁/ ▁ xxup ▁ g pl ▁. ▁ xxmaj ▁porque ▁la ▁empresa ▁hace ▁también ▁software ▁que ▁se ▁bloque en ▁después ▁de ▁10 ▁minutos ▁para ▁que ▁la ▁gente
4,"▁ xxmaj ▁quizá ▁habría ▁que ▁trasladar lo ▁de ▁momento ▁a ▁ xxmaj ▁intento ▁de ▁ xxmaj ▁golpe ▁de ▁ xxmaj ▁estado ▁en ▁ xxmaj ▁turquía ▁de ▁2016 ▁, ▁pero ▁habría ▁que ▁borra r ▁la ▁re di rec ción ▁que ▁he ▁creado ▁. ▁18 p x ▁ xxmaj ▁me ▁parece ▁bien ▁/ ▁ xxmaj ▁salud os ▁18 p x ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁hijo ▁de ▁pu ta ▁,"


In [ ]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(1, 1e-4 * 10, moms=(0.8, 0.7))
# learn.unfreeze()
# learn.fit_one_cycle(10, 1e-4, moms=(0.8, 0.7))

In [ ]:
path='/content/gdrive/My Drive/multilingual toxic/es/es1'
self=exp.finetune_lm
CLS_BEST = path+'/cls_best1'
LM_BEST = path+"/lm_best1"
ENC_BEST = path+"/enc_best1"

In [ ]:
path='/content/gdrive/My Drive/multilingual toxic/es/es1'
self=exp.finetune_lm
CLS_BEST = path+'/cls_best'
LM_BEST = path+"/lm_best"
ENC_BEST = path+"/enc_best"


experiment_path=Path(path)
self.experiment_path=experiment_path
print("Experiment", experiment_path)

#save tokenizer
tokenizer.save(self.experiment_path, learn=learn)
learn.to_fp32()
#save encoder weights to (ENC_BEST,learn.path,learn.model_dir)
learn.save_encoder(ENC_BEST)
#save model structure to (LM_BEST, learn.path,learn.model_dir,'.pth')
learn.save(LM_BEST, with_opt=False)
# learn.destroy()
# saving 'seed', 'name', 'arch', 'experiment_path', 'dataset_path', 'num_epochs', 'bs', 'bptt', 'drop_mult', 'dropout_values', 'label_smoothing_eps', 'label_smoothing_eps_norm_by_classes', 'use_adam_08', 'true_wd', 'wd', 'clip', 'fp16', 'lr', 'base' 
# to /content/gdrive/My Drive/multilingual toxic/es/es1/finetuning.json
# save_paramters(self)

Experiment /content/gdrive/My Drive/multilingual toxic/es/es1
Copy sp model from /root/.fastai/models/es_multifit_paper_version to /content/gdrive/My Drive/multilingual toxic/es/es1


In [ ]:
self=exp.classifier
es['toxic']=es['toxic'].astype(np.long)
from pathlib import Path
data_clas = (TextList.from_df(es,cols=['content','toxic'], **fa_config)
            .split_by_rand_pct(0.1)
            .label_from_df('toxic')           
            .databunch(bs=32))
l1rn = exp.classifier.get_learner(data_clas)  
# learn is a preconfigured fastai learner with a pretrained model loaded
data_clas.show_batch()

Using Label smoothing with eps =  0.05
Setting Classifier weights seed seed to 0
Training args:  {'drop_mult': 0.5, 'wd': 0.01, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy(), 'clip': 0.12, 'silent': False} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15}
Loading pretrained model /content/gdrive/My Drive/multilingual toxic/es/es1/enc_best
Setting Classifier training seed seed to 0


text,target
▁ xxbos ▁ xxfld ▁1 ▁http ▁: ▁/ ▁/ ▁www . the - af c . com ▁/ ▁en ▁/ ▁e vent - in form a tion ▁/ ▁ af c - asi an - cu p - ma tch - off icia l s ▁http ▁: ▁/ ▁/ ▁www . the - af c . com ▁/ ▁en ▁/ ▁compone nt ▁/ ▁ jo om lea gue ▁/ ▁,0
▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁ xxup ▁san ▁ xxup ▁ ba udi lio ▁ n ▁ xxrep ▁ 44 ▁o ▁,0
"▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁señor ▁ xxup ▁ iva n ▁ xxup ▁ ce pe da ▁, ▁ xxup ▁duro ▁ xxup ▁ cri tico ▁ xxup ▁del ▁ xxup ▁mejor ▁ xxup ▁presidente ▁ xxup ▁que ▁ xxup ▁he mos ▁ xxup ▁tenido s ▁ xxup ▁los ▁ xxup ▁colombiano s ▁ xxup ▁ vic tima ▁ xxup ▁de ▁ xxup ▁las ▁ xxup ▁a tro",1
"▁ xxbos ▁ xxfld ▁1 ▁no ▁mas ▁de s on rra s ▁chica s ▁... ▁re be nte mos ▁una ▁re pres a ▁, ▁como ▁, ▁ ra pel ▁, ▁mau le ▁. . ▁ oh ▁haciendo la ▁ xxup ▁esta lla r ▁... ▁ xxup ▁ca os ▁ xxup ▁total ▁ xxup ▁en ▁ xxup ▁chile ▁... ▁ xxup ▁honra ▁ xxup ▁ ah ▁ xxup ▁nivel ▁ xxup",1
"▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁cada ▁ xxup ▁ dia ▁ xxup ▁es ▁ xxup ▁mas ▁ xxup ▁evidente ▁ xxup ▁que ▁ xxup ▁el ▁ xxup ▁pueblo ▁ xxup ▁colombiano ▁ xxup ▁rechaza ▁ xxup ▁cada ▁ xxup ▁vez ▁ xxup ▁mas ▁ xxup ▁los ▁ xxup ▁actos ▁, ▁ xxup ▁poli tica s ▁e ▁ xxup ▁ ide ologi a ▁ xxup ▁po dri da ▁",0


In [ ]:
l1rn.unfreeze()
def _fit_schedule_1cycle(num_epochs, learn):
        learn.unfreeze()
        learn.fit_one_cycle(num_epochs, slice(1e-2 / (2.6 ** 4), 2e-2), moms=(0.8, 0.7))

def _fit_schedule_layered(num_epochs, learn):
        learn.freeze_to(-1)
        learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))
        if num_epochs > 1:
            learn.freeze_to(-2)
            learn.fit_one_cycle(1, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7))
            learn.freeze_to(-3)
            learn.fit_one_cycle(1, slice(5e-3 / (2.6 ** 4), 5e-3), moms=(0.8, 0.7))
            learn.unfreeze()
        if num_epochs > 5:
            learn.fit_one_cycle(num_epochs - 4, slice(1e-3 / (2.6 ** 4), 1e-3), moms=(0.8, 0.7))

def _fit_schedule_2cycle(num_epochs, learn):
        learn.freeze_to(-1)
        learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))
        learn.unfreeze()
        if num_epochs > 1:
            learn.fit_one_cycle(num_epochs - 1, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7))

def _fit_schedule_reverse_2cycle(num_epochs, learn):
        learn.unfreeze()
        for g in learn.layer_groups[-1:]:
            for l in g:
                if not learn.train_bn or not isinstance(l, bn_types): requires_grad(l, False)
        learn.create_opt(defaults.lr)
        learn.fit_one_cycle(num_epochs, slice(1e-2 / (2.6 ** 4), 2e-2), moms=(0.8, 0.7))
        learn.unfreeze()
        learn.fit_one_cycle(num_epochs, slice(1e-3 / (2.6 ** 4), 2e-3), moms=(0.8, 0.7))

def _fit_schedule_false_wd(self, learn):
        learn.true_wd = False
        learn.fit_one_cycle(1, 5e-2, moms=(0.8, 0.7), wd=1e-7)
        if num_epochs > 1:
            learn.freeze_to(-2)
            learn.fit_one_cycle(1, slice(5e-2 / (2.6 ** 4), 5e-2), moms=(0.8, 0.7), wd=1e-7)
            learn.freeze_to(-3)
            learn.fit_one_cycle(1, slice(5e-4 / (2.6 ** 4), 5e-4), moms=(0.8, 0.7), wd=1e-7)
            learn.unfreeze()
            if num_epochs > 5:
                learn.fit_one_cycle(num_epochs - 4, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7), wd=1e-7)

# _fit_schedule_1cycle(10,l1rn)

In [ ]:
self.experiment_path = l1rn.path / l1rn.model_dir
tokenizer.save(self.experiment_path, learn=l1rn)
l1rn.to_fp32()
l1rn.save(CLS_BEST, with_opt=False)
print("Classifier model saved to", self.experiment_path)
self.save_paramters()
        # learn.destroy()

Copy sp model from /root/.fastai/models/es_multifit_paper_version to multifit_paper_version
Classifier model saved to multifit_paper_version
Saving dump to multifit_paper_version/classifier.json


'{\n  "seed": 0,\n  "name": "multifit_paper_version",\n  "arch": {\n    "tokenizer_type": "sp",\n    "max_vocab": 15000,\n    "lang": "es",\n    "emb_sz": 400,\n    "n_hid": 1550,\n    "n_layers": 4,\n    "qrnn": true\n  },\n  "experiment_path": "multifit_paper_version",\n  "dataset_path": null,\n  "bs": 18,\n  "num_epochs": 8,\n  "drop_mult": 0.5,\n  "dropout_values": {\n    "output_p": 0.25,\n    "hidden_p": 0.1,\n    "input_p": 0.2,\n    "embed_p": 0.02,\n    "weight_p": 0.15\n  },\n  "wd": 0.01,\n  "clip": 0.12,\n  "label_smoothing_eps": 0.1,\n  "label_smoothing_eps_norm_by_classes": true,\n  "weighted_cross_entropy": null,\n  "early_stopping": null,\n  "fit_schedule": "1cycle",\n  "random_init": false,\n  "bptt": 70,\n  "fp16": false,\n  "base": "/content/gdrive/My Drive/multilingual toxic/es/es1"\n}'